# TP1 - Exercício 1
## Grupo 1

*   Diogo Coelho da Silva A100092
*   Pedro Miguel Ramôa Oliveira A97686

**Problema proposto:**

1.  Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:
    - **a)** Cada reunião ocupa uma sala (enumeradas 1...S) durante um “slot” (tempo,dia).  Assume-se os dias enumerados 1..D e, em cada dia, os tempos enumerados 1..T.
    - **b)** Cada reunião tem associado um projeto (enumerados 1..P) e um conjunto de participantes. Os diferentes colaboradores são enumerados 1..C.
    - **c)** Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. São “inputs” do problema o conjunto de colaboradores de cada projeto, o seu líder e o número de reuniões semanais.
    - **d)** O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  50% do total de colaboradores do projeto.  A disponibilidade de cada participante, incluindo o lider,  é um conjunto de “slots” (“inputs” do problema).

**Proposta de resolução:**

O problema que foi apresentado tem como objetivo a criação de um horário semanal otimizado para a marcação de reuniões para uma *startup*. O objetivo é alocar eficientemente recursos como salas e tempo, considerando a disponibilidade dos colaboradores e as restrições dadas no enunciado.

Na solução proposta iremos modelar o problema de acordo com as nossas necessidades e utilizaremos um solver SCIP para encontrar uma solução otimizada.

Foram consideradas restrições dadas pelo enunciado, como por exemplo, a obrigatoriedade da presença do lider em todas as reuniões do seu projeto, a disponibilidade dos colaboradores, a partição exclusiva dos membros do projeto, a alocação de apenas um projeto por sala em cada slot e a garantia de um quorum minimo em cada reunião.









#### 1. Importar as bibliotecas importantes

In [18]:
from ortools.linear_solver import pywraplp
import random
import pandas as pd

- **pywraplp**: Importa o solver de programação linear da biblioteca OR-Tools para resolver o problema de otimização.
- **random**: Gera aleatoriedade para simular a disponibilidade de colaboradores e equipes dos projetos.
- **pandas**: Biblioteca usada para organizar os dados em formato de tabela (DataFrame), facilitando a manipulação e visualização.

#### 2. Pedir variáveis de entrada ao utilizador

In [19]:
#Função que recolhe as variáveis do utilizador... tem valores padrão pré-definidos no caso do utilizador não introduzir valores
def get_input(prompt, default):
    user_input = input(f"{prompt} [{default}]: ")  # Mostra o valor padrão
    return int(user_input) if user_input else default  # Usa o valor padrão se o input estiver vazio

S = get_input("Digite o número de salas", 5)
D = get_input("Digite o número de dias", 5)
H = get_input("Digite o número de slots por dia", 8)
P = get_input("Digite o número de projetos", 5)
C = get_input("Digite o número de colaboradores", 30)

#Pretty print das variáveis introduzidas pelo utilizador
print(f"Parâmetros recebidos: {S} salas, {D} dias, {H} slots por dia, {P} projetos, {C} colaboradores.")

Parâmetros recebidos: 5 salas, 5 dias, 8 slots por dia, 5 projetos, 30 colaboradores.


Nesta parte do código são definidas as variáveis do problema. A função definida "get_input" pergunta ao utilizador os valores que pretende introduzir para o problema. Se o utilizador não colocar valores, existem valores pre-definidos pelo programa. Existe também controlo de erros, como por exemplo, a eventualidade de o introduzir valores não validos, como por exemplo uma string.
As variáveis recolhidas são as seguintes: 
- **S** -> representa o número de salas (valor default = 5)
- **D** -> representa o número de dias (valor default = 5)
- **H** -> representa o número de horas (valor default = 8)
- **P** -> representa o número de projetos (valor default = 5)
- **C** -> representa o número de colaboradores (valor default = 30)


DEFINIR SLOTS E COLABORES

In [20]:
# Definir slots e colaboradores

#Define as slots de time em pares(dias,horas)
Slots = [(d, h) for d in range(D) for h in range(H)]
#Define os colaboradores em set para nao existirem repetidos
Colabs = set(range(1, C + 1)) 

random.seed(42) 
#TO-DO
Colaboradores = [random.sample(Slots, 20) for _ in range(1, C + 1)]

'''
Primeiro é inicializado um array vazio para guardar os projetos. De seguida, enquanto que existirem projetos disponiveis, percorremos um ciclo e vamos
atribuindo colaboradores de forma random aos projetos
'''
Projectos = []
for _ in range(P):
    available_team_size = min(3, len(Colabs))  #Não pode ter menos do que 3 colaboradores
    team = random.sample(list(Colabs), available_team_size)
    Projectos.append((random.randint(1, 5), team))
    Colabs = Colabs - set(team)

INICIALIZAR O SOLVER

In [21]:
# Inicializando o solver SCIP
horario = pywraplp.Solver.CreateSolver('SCIP')
# Variáveis de decisão: x[s, d, h, p, c] = 1 se o colaborador c participa na reunião do projeto p no slot (sala s, dia d, hora h)
x = {}
for s in range(S):
    for d in range(D):
        for h in range(H):
            for p in range(P):
                for c in range(1, C + 1):  # Fix: Range from 1 to C
                    x[s, d, h, p, c] = horario.BoolVar(f'x[{s},{d},{h},{p},{c}]')


RESTRIÇÃO 1

In [22]:

# Restrição 1: O líder tem de estar em todas as R reuniões do seu projecto
for p in range(P):
    lider = Projectos[p][1][0]
    horario.Add(
        sum(x[s, d, h, p, lider] for s in range(S) for d in range(D) for h in range(H)) == Projectos[p][0]
    )

RESTRIÇÃO 2

In [23]:
# Restrição 2: Colaboradores fora da disponibilidade não podem participar
for s in range(S):
    for d in range(D):
        for h in range(H):
            for p in range(P):
                for c in range(1, C + 1):  # Fix: Range from 1 to C
                    if (d, h) not in Colaboradores[c-1]:  # Fix: Adjust index since Colaboradores uses 0-based indexing
                        horario.Add(x[s, d, h, p, c] == 0)


RESTRIÇÃO 3

In [24]:
# Restrição 3: Apenas colaboradores do projeto podem participar
for s in range(S):
    for d in range(D):
        for h in range(H):
            for p in range(P):
                for c in range(1, C + 1):  # Fix: Range from 1 to C
                    if c not in Projectos[p][1]:
                        horario.Add(x[s, d, h, p, c] == 0)


RESTRIÇÃO 4

In [25]:
# Restrição 4: Uma sala pode ter apenas um projeto por slot de tempo
for s in range(S):
    for d in range(D):
        for h in range(H):
            horario.Add(sum(x[s, d, h, p, Projectos[p][1][0]] for p in range(P)) <= 1)


RESTRIÇÃO 5

In [26]:
# Restrição 5: Um colaborador só pode estar numa sala em um dado slot
for d in range(D):
    for h in range(H):
        for p in range(P):
            for c in Projectos[p][1]:
                horario.Add(sum(x[s, d, h, p, c] for s in range(S)) <= 1)


RESTRIÇÃO 6

In [27]:
# Restrição 6: Quorum de 50% dos colaboradores, incluindo o líder
for s in range(S):
    for d in range(D):
        for h in range(H):
            for p in range(P):
                quorum = len(Projectos[p][1]) // 2 + 1  # 50% ou mais com o líder
                horario.Add(sum(x[s, d, h, p, c] for c in Projectos[p][1]) >= quorum * x[s, d, h, p, Projectos[p][1][0]])


FUNCAO OBJETIVO

In [28]:
horario.Maximize(
    sum(x[s, d, h, p, c] for s in range(S) for d in range(D) for h in range(H) for p in range(P) for c in range(1, C + 1))
)

SOLVER

In [29]:
# Resolver o problema
status = horario.Solve()

In [30]:




'''
# Mapeamento dos slots para horas
time_slots = ["08:30 - 09:30", "09:30 - 10:30", "10:30 - 11:30", "11:30 - 12:30",
              "13:30 - 14:30", "14:30 - 15:30", "15:30 - 16:30", "16:30 - 17:30"]
'''


# Verificar a solução e preparar a tabela de resultados
schedule_data = []


if status == pywraplp.Solver.OPTIMAL:
    for p in range(P):
        for s in range(S):
            for d in range(D):
                for h in range(H):
                    if round(x[s, d, h, p, Projectos[p][1][0]].solution_value()) == 1:
                        participants = [c for c in Projectos[p][1] if round(x[s, d, h, p, c].solution_value()) == 1]
                        # Combine day and hour as one column, with hour as a simple number
                        schedule_data.append([p + 1, s + 1, f"{d + 1},{h + 1}", participants])
else:
    # Ensure this matches the number of columns (4 columns)
    schedule_data.append([None, None, None, "Nenhuma solução ótima encontrada."])

# Criar dataframe da tabela
df_schedule = pd.DataFrame(schedule_data, columns=["Projeto", "Sala", "Dia_Hora", "Participantes"])


# Criar estilo para linhas alternadas e bordas entre colunas
def style_schedule(df):
    return df.style.set_table_styles(
        [{'selector': 'tr:nth-child(even)',
          'props': [('background-color', '#f2f2f2')]},   # Cor alternada para linhas pares
         {'selector': 'th, td',
          'props': [('border', '1px solid black')]}      # Bordas em volta das células
        ]
    ).set_properties(**{'text-align': 'center'})          # Alinhar o texto no centro

# Aplicar estilo e exibir diretamente no notebook
styled_table = style_schedule(df_schedule)

# Exibir a tabela estilizada no Jupyter Notebook sem o índice
display(styled_table.hide(axis='index'))


Projeto,Sala,Dia_Hora,Participantes
1,1,"1,4","[24, 16]"
1,1,"1,6","[24, 28, 16]"
1,1,"4,2","[24, 28, 16]"
1,1,"4,3","[24, 16]"
2,1,"1,1","[9, 27, 8]"
2,1,"3,5","[9, 27, 8]"
2,1,"3,8","[9, 27, 8]"
3,1,"2,3","[20, 19]"
3,1,"3,1","[20, 25]"
4,1,"1,2","[11, 3]"
